In [2]:
!pip install earthengine-api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 3.4 MB/s eta 0:00:0000:0100:01


In [4]:
import ee
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [10]:
import ee
ee.Initialize()

# Define a region of interest
roi = ee.Geometry.Point([-122.262, 37.8719])

# Load Landsat 8 image collection, filter by date and region
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_TOA").filterDate('2020-01-01', '2020-12-31').filterBounds(roi)

# Select specific bands (e.g., B4 (red), B5 (near-infrared))
bands = landsat.select(['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11'])

# Get the first image
image = bands.first()

# Print image info
print(image.getInfo())


{'type': 'Image', 'bands': [{'id': 'B1', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [7661, 7801], 'crs': 'EPSG:32610', 'crs_transform': [30, 0, 464685, 0, -30, 4264515]}, {'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [7661, 7801], 'crs': 'EPSG:32610', 'crs_transform': [30, 0, 464685, 0, -30, 4264515]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [7661, 7801], 'crs': 'EPSG:32610', 'crs_transform': [30, 0, 464685, 0, -30, 4264515]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [7661, 7801], 'crs': 'EPSG:32610', 'crs_transform': [30, 0, 464685, 0, -30, 4264515]}, {'id': 'B5', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [7661, 7801], 'crs': 'EPSG:32610', 'crs_transform': [30, 0, 464685, 0, -30, 4264515]}, {'id': 'B6', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [7661, 7801], 'crs': 'EPSG:

In [7]:
!pip install folium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 2.2 MB/s eta 0:00:0000:0100:01


In [11]:
import folium

vis_params = {
    'min': 0,
    'max': 0.3,
    'bands': ['B9', 'B10', 'B11'],
}

# Create a folium map centered at the region of interest
map_center = [37.8719, -122.262]
m = folium.Map(location=map_center, zoom_start=12)

# Add the image layer to the map
map_id_dict = ee.Image(image).getMapId(vis_params)
folium.TileLayer(
    tiles=map_id_dict['tile_fetcher'].url_format,
    attr='Google Earth Engine',
    name='Landsat 8 True Color',
    overlay=True,
    control=True
).add_to(m)

# Add layer control
m.add_child(folium.LayerControl())

# Display the map
m